# Instructions
The following code was designed to help select parameters for each channel to be used when running CellCounter.ipynb.  On a sample/composite image for which cells have been manually counted (instructions below) an iterative series of thresholds are applied before automatic counting is performed and the results for each threshold can then be compared to the manual counts. The goal is to select thresholds that yield the greatest consistency with manual counts.  Consistency is based upon the number of counted cells and their locations.   

*Note that this procedure must be performed separately for each channel to be counted.


### The data returned
This code will return a summary file with the following info for each threshold applied: the threshold applied, the Otsu threshold, the number of manually counted cells, the avg cell diameter specified by the user, the number of automatically counted cells, the number of automatically counted cells that correspond to the location of a single manually counted cell (Hits), and the average size of automatically counted cells (in pixels).  Additionally, two measures of accuracy are returned: the number of hits divided by the number of manually counted cells, and the number of hits divided by the number of automatically counted cells.

*The Otsu threshold defines a threshold that optimally separates background from signal (http://www.labbookpages.co.uk/software/imgProc/otsuThreshold.html).  This will not necessarily be the threshold that one chooses, but is good to use as a comparison.  One may pick a threshold that is more stringent than OTSU (ie, higher than OTSU), in order to select cells that are more prominent.  


### Choosing a threshold
An optimal threshold should:

1 - Identify the location of a large number of the manually counted cells.

2 - Have a relatively small number of automatically counted cells that are not manually counted (i.e. few false positives).

3 - Separate signal from background.  

The first criterion can be evaluated by examining the output variable Hits/Manually Counted Cells.  This is the proportion of manually counted cells that the counter identifies the location of.  In practice, it is easy to get values above .8-.90.  However, it can be the case that with a given threshold the counter correctly marks 100% of manually counted cells but also marks many more spots where no cells are present. Hence the second criterion.

The second criterion is harder to judge but can be evaluated by examining the output variable Hits/Automatically Counted Cells.  That is, what is the proportion of cells judged by the counter to be cells that were also judged manually to be cells?  It is important to note that there are several factors that can reduce this proportion beyond the program calling things cells that aren't cells.  First, it is possible that a true cell was not marked by the experimenter and the program is finding these cells.  Second, because thresholding reduces the apparent size of an identified cell, and because the marked location of the manual counts is a single pixel, it is possible that the manually marked location of a cell is just outside of the automatically marked location of a cell.  Because of this, the returned value is a lower bound on accuracy.  Thus, if this value is 0.8, the upper bound for false positives is 0.2. 

Currently, I think it should be possible to choose a threshold with >0.8 Hits/Manual Counts and >0.7 for Hits/Automatic Counts.  I have been taking the average of these two and selecting the threshold for which this average is highest.

Lastly, the Otsu method is used to select a threshold that optimally separates signal from noise based upon the distribution of grayscale intensities.  While this generally does a pretty good job, it often seems that a more stringent criterion is helpful to reduce false positives.  In light of this, I think that the Otsu threshold might be the lower bound for what one might choose.




### Creating a composite image
A composite image is made up of  multiple images stitched together side by side (e.g. using ImageJ).  In this way, the chosen parameters are not based upon a single image from one animals.  I personally take 1-2 images from animals in each group and stitch them together. Alternatively, if each image has many cells, it can be easier to do the same thing but take some portion of each image (e.g. 100x100 pixels from each 1000x1000 image).  The composite image should be a .tif image.  If optimizing multiple channels, the composite image for each channel does not need to overlap with that of the other channel. 


### Creating a mask
Regions of interest are best drawn using ImajeJ.  Binary masks are created such that the region of interest pixel values = 255, whereas other values are equal to 0
#1 - Open the image in ImageJ
#2 - Use your preferred selection tool (here, multipoint selection) to mark the cells of interest
#3 - Go to edit > selection > create mask
#4 - Save the mask as .tif file


### Manually counting cells from the composite image / Creating a mask
Using ImageJ, the center of each cell in the composite is marked using the multipoint selection tool and this selection is saved as a mask (the center of each cell is set to 255 and all other locations are set to 0).  The mask should be saved as a .tif image.
*A note on counting cells manually: I typically opt for counting cells that are bright and in focus, while avoiding counting cells that are difficult to make a judgment about.  Additionally, try to center your count locations  in the middle of the brightest portion off your cell.  Because thresholding makes cells smaller in practice, and the location of the manual count is 1 pixel, marking cells in this way maximizes accuracy.


### Folder Structure
For this code to work files should be organized in a specific manner.
Directory_Main (the path of which is to be defined by the user... actual directory name can be whatever you want) should contain the subfolders "Composite","ManualCounts", and "SavedOutput" (these names matter).  

Composite and ManualCounts folders should contain .tif images with a single composite image and mask, respectively. 
Example filenames under Composite and ManualCounts: "Study1_Channel_1_Composite.tif","Study1_Channel_1_Composite_Mask.tif"


### Requirements
The following will need to be installed in your Conda environment:

1 - python (3.6.5)

2 - jupyter

3 - imread

4 - mahotas(1.4.4)

5 - numpy(1.14.3)

6 - pandas(0.23.0)

7 - matplotlib(2.2.2). 

The following commands can be executed in your terminal to create the environment: 

conda config --add channels conda-forge

conda create -n EnvironmentName python=3.6.5 mahotas=1.4.4 pandas=0.23.0 matplotlib=2.2.2 jupyter imread

Different versions of these packages have not been tested.  Single channel .tif images are required (some image acquisition software exports single channel images with more than >2 dimensions.  If this is the case, images must first be converted to 2 dimensional space).  Images that are not already 8-bit are converted to 8 bit.  However, these images are not necessarily on a scale from 0-255.  It is therefore advised that all images are set to 8 bit before running this.  If this was not done, be sure that composite image running during optimization is of the exact same form.  Additionally, for measuring diameter this should be in pixel units.  To insure files are in pixel scale, in ImageJ go to Analyze -> SetScale -> Click to Remove Scale -> OK.

### What type of cells can this count?
Any semi-circular cell that is more or less filled by the fluorescent label will be capable of being counted.  If the cells are only visible by their perimeter than this code as is may not work. In this case, a process should be applied to either a) fill the internal component of the cell after thresholding or b) use smoothing or morphological opening to increase the internal of the cell. 

### Viewing an example of how images are being processed
In order to view a single threshold looks like, one can go to the last cell, entitled "Display Result From Single Threshold".  To run this cell of code only the cells up to and including 'Function to Count Cells' must be run


# Specify Parameters and Options for Running
### This should be the only part of the code that is required to be changed


In [ ]:
#Define directory of working environment (Directory.Main)
Directory_Main = "/Users/ZP/Images"

#Set Parameters and Options
CellDiam = 10 #Average Cell Diameter in pixel units.  Must be integer value.  
UseWatershed = True #Perform watershed for this channel? True/False
particle_minimum = 0.2 #After thresholding this serves to erase excessively small points.  Proportion of average cell size permitted.  Average cell size is assumed to be square of diamater for rough approximation

#
#Do not change
#
UseROI = False
#Because composite does not utilize ROI always have this set to False

# Load Necessary Packages

In [ ]:
import pylab
import os
import fnmatch
import imread
import numpy as np
import pandas as pd
import mahotas as mh
import matplotlib as mpl
import matplotlib.pyplot as plt

# Get Directory Information

In [ ]:
#Define existing subdirectories
Directory_Composite = Directory_Main + "/" + "Composite"
Directory_Manual = Directory_Main + "/" + "ManualCounts"
Directory_Output = Directory_Main + "/" + "SavedOutput"


#Get filenames for composite image and mask of cell locations from manual counting
FileName_Composite = sorted(os.listdir(Directory_Composite)) #get list of files in composite folder.  there should only be 1
FileName_Composite = fnmatch.filter(FileName_Composite, '*.tif') #restrict files in folder to .tif files
FileName_Manual = sorted(os.listdir(Directory_Manual)) #get list of files in composite folder.  there should only be 1
FileName_Manual = fnmatch.filter(FileName_Manual, '*.tif') #restrict files in folder to .tif files


# Get Manual Counts and Display Initial Processing of Image

In [ ]:
#Get manual counts
Image_Manual = mh.imread((Directory_Manual + "/" + FileName_Manual[0]),as_grey=True)#Load File as greyscale image
ManuallyCounted = (Image_Manual>0).sum()

#Load Composite Image
Image_Composite = mh.imread((Directory_Composite + "/" + FileName_Composite[0]),as_grey=True) #Load File as greyscale image
if Image_Composite.dtype!="uint8": #Check if composite image is uint8
    Image_Composite=Image_Composite.astype('uint8')
    print("Warning: Converted "+FileName_Composite+" to uint8.  See notes in structions")
else:    
    pass

#Substract Background
Image_Composite_BG = mh.gaussian_filter(Image_Composite,sigma=CellDiam*3) #Determine regional background
Image_Composite_BG = Image_Composite - Image_Composite_BG #Subtract background from orginal image
Image_Composite_BG[Image_Composite_BG<0] = 0 #Set negative values = 0

#Apply Gaussian Filter to Image
Image_Composite_Gaussian = mh.gaussian_filter(Image_Composite_BG,sigma=CellDiam/12) #Currently, cell diameter is 12 standard deviations

#Obtain Otsu threshold
OTSU = mh.otsu(Image_Composite_Gaussian.astype("uint8"))
Image_OTSU = Image_Composite_Gaussian > OTSU

#Display Image
plt.gray()
plt.figure(figsize=(20,20))

plt.subplot(1,3,1)
plt.title('Original Image')
plt.imshow(Image_Composite)
plt.subplot (1,3,2)
plt.title('Preprocessed Image')
plt.imshow(Image_Composite_Gaussian)
plt.subplot (1,3,3)
plt.title('OTSU Threshold: ' + str(OTSU))
plt.imshow(Image_OTSU)


# Load Function

## Function to Count Cells

In [ ]:
#Function to Count Cells
def Count(CellDiam,Thresh,Directory_Current,FileNames_Current,UseWatershed,x):
    
    #Load file
    Image_Current_File = Directory_Current + "/" + FileNames_Current[x] #Set directory location
    Image_Current_Gray = mh.imread(Image_Current_File,as_grey=True) #Load File as greyscale image
    print("Processing: " + FileNames_Current[x])

    #Convert image to uint8 if it isn't already
    if Image_Current_Gray.dtype!="uint8": 
        Image_Current_Gray=Image_Current_Gray.astype('uint8')
        print("Warning: Converted "+FileNames_Current[x]+" to uint8.  See notes in structions")
    else:    
        pass

    #Substract Background
    Image_Current_BG = mh.gaussian_filter(Image_Current_Gray,sigma=CellDiam*3) #Determine regional background
    Image_Current_BG = Image_Current_Gray - Image_Current_BG #Subtract background from orginal image
    Image_Current_BG[Image_Current_BG<0] = 0 #Set negative values = 0

    #Apply Gaussian Filter to Image
    Image_Current_Gaussian = mh.gaussian_filter(Image_Current_BG,sigma=CellDiam/12) #Currently, cell diameter is 12 standard deviations

    #Threshold image 
    Image_Current_T = Image_Current_Gaussian > Thresh #Threshold image

    #Erase any particles that are below the minimum particle size
    labeled,nr_objects = mh.label(Image_Current_T) #label particles in Image_Current_T
    sizes = mh.labeled.labeled_size(labeled) #get list of particle sizes in Image_Current_T
    too_small = np.where(sizes < (CellDiam*CellDiam*particle_minimum)) #get list of particle sizes that are too small
    labeled = mh.labeled.remove_regions(labeled, too_small) #remove particle sizes that are too small for labeling of Image_Current_T
    Image_Current_T = labeled != 0 #reconstitute Image_Current_T with particles removed

    #Get ROI and apply to thresholded image
    if UseROI:
        ROI_Current_File = Directory_ROI + "/" + FileNames_ROI[x] #Set directory location of ROI file
        ROI_Current = mh.imread(ROI_Current_File,as_grey=True) #Load File
        Image_Current_T[ROI_Current==0]=0 #Set values of thresholded image outside of ROI to 0
        roi_size = np.count_nonzero(ROI_Current)
    else:
        roi_size = Image_Current_Gray.size

    #Count cells and find locations
    if UseWatershed:

        #If there are pixels above the threshold proceed to watershed
        if Image_Current_T.max() == True:

            #Create distance transform from thresholded image to help identify cell seeds
            Image_Current_Tdist = mh.distance(Image_Current_T)

            #Define Sure Background for watershed
            #Background is dilated proportional to cell diam.  Allows final cell sizes to be a bit larger at end.  Will not affect cell number but can influence overlap
            #See https://docs.opencv.org/3.4/d3/db4/tutorial_py_watershed.html for tutorial that helps explain this
            Dilate_Iterations = int(CellDiam//2) 
            Dilate_bc = np.ones((3,3)) #Use square structuring element instead of cross
            Image_Current_SureBackground = Image_Current_T
            for j in range (Dilate_Iterations): 
                Image_Current_SureBackground = mh.dilate(Image_Current_SureBackground,Bc=Dilate_bc)

            #Create seeds/foreground for watershed
            #See https://docs.opencv.org/3.4/d3/db4/tutorial_py_watershed.html for tutorial that helps explain this
            Regmax_bc = np.ones((CellDiam,CellDiam)) #Define structure element regional maximum function.  Currently uses diamater
            Image_Current_Seeds = mh.regmax(Image_Current_Tdist,Bc=Regmax_bc) #Find regional maxima of distance transform
            seeds,nr_nuclei = mh.label(Image_Current_Seeds)

            #Define unknown region between sure foreground (the seeds) and sure background 
            Image_Current_Unknown = Image_Current_SureBackground.astype(int) - Image_Current_Seeds.astype(int)

            #Modify seeds to differentiate between background and unknown regions
            seeds+=1
            seeds[Image_Current_Unknown==1]=0

            #Perform watershed
            Image_Current_Watershed = mh.cwatershed(surface=Image_Current_SureBackground,markers=seeds)
            Image_Current_Watershed -= 1 #Done so that background value is equal to 0.
            Image_Current_Cells = Image_Current_Watershed

        #If there are no pixels above the threshold watershed procedure has issues.  Set cell count to 0.
        elif Image_Current_T.max() == False:
            Image_Current_Cells = Image_Current_T.astype(int)
            nr_nuclei = 0

    else:
        #Method for counting cells when watershed is not selected
        Image_Current_Cells, nr_nuclei = mh.label(Image_Current_T)

        
    return Image_Current_Cells, nr_nuclei, roi_size, Image_Current_Gray, Image_Current_Gaussian, Image_Current_T

# Iterate Through Threshold Values and Count

In [ ]:
#Initialize Arrays to Store Data In
List_AutoCounts = []
List_Hits = []
List_CellAreas = []
List_Acc_HitsOverAutoCounts = []
List_Acc_HitsOverManualCounts = []

#Define maximum threshold value and create series of thresholds to cycle through
TMax = int(Image_Composite_Gaussian.max()//1) #Get maximum value in array.  Threshold can't go beyond this
List_ThreshValues = list(range(1,TMax))


#Iterate through possible thresholds
for Thresh in List_ThreshValues:
    
    print("Counting Threshold " + str(Thresh))
    
    #Call function to count cells with given threshold parameter and save cell counts
    Image_Current_Cells, nr_nuclei, roi_size, Image_Current_Gray, Image_Current_Gaussian, Image_Current_T = Count(CellDiam,Thresh,Directory_Composite,FileName_Composite,UseWatershed,0) #Last index specifies location in file
    List_AutoCounts.append(nr_nuclei)
    
    #Determine Avg Cell Size in Pixel Units
    if nr_nuclei > 0:
        Cell_Area = Image_Current_Cells > 0
        Cell_Area = Cell_Area.sum() / nr_nuclei
    elif nr_nuclei == 0:
        Cell_Area = float('nan')
    List_CellAreas.append(Cell_Area)
    
    #Determine Number of Automatically Counted Cells Whose Location Corresponds to a Manually Counted Cell
    Hits = 0 #Initialize starting value of hits
    if nr_nuclei > 0:
        for x in range (1,nr_nuclei+1): #for each automatically counted cell
            HitMap = Image_Manual[Image_Current_Cells==x]
            if HitMap.sum()==255:
                Hits += 1
            else:
                pass
    else:
        pass
    List_Hits.append(Hits)
    
    #Calculate Accuracies
    try:
        List_Acc_HitsOverAutoCounts.append(Hits/nr_nuclei)
    except: #if divide by zero occurs, set value to nan
        List_Acc_HitsOverAutoCounts.append(np.nan)
        
    List_Acc_HitsOverManualCounts.append(Hits/ManuallyCounted)

# Create DataFrame Summary and Save

In [ ]:
#Create Dataframe
DataFrame = pd.DataFrame(
    {'AutoCount_Thresh': List_ThreshValues,
     'OTSU_Thresh': np.ones(len(List_ThreshValues))*OTSU,
     'Manual_CellDiam': np.ones(len(List_ThreshValues))*CellDiam,
     'Manual_Counts': np.ones(len(List_ThreshValues))*ManuallyCounted,
     'AutoCount_UseWatershed': np.ones(len(List_ThreshValues))*UseWatershed,
     'AutoCount_Counts': List_AutoCounts,
     'AutoCount_Hits': List_Hits,
     'AutoCount_AvgCellArea': List_CellAreas,
     'Acc_HitsOverManualCounts': List_Acc_HitsOverManualCounts,
     'Acc_HitsOverAutoCounts': List_Acc_HitsOverAutoCounts,
     'Acc_HitsOverManualCounts': List_Acc_HitsOverManualCounts,
    })   

DataFrame.to_csv(Directory_Output + "/" + "Thresh_Summary.csv")

# Display result from single threshold
## Here user can set threshold to any desired value and results will be displayed.

In [ ]:
Thresh = 81

#Call function to count cells
Image_Current_Cells, nr_nuclei, roi_size, Image_Current_Gray, Image_Current_Gaussian, Image_Current_T = Count(CellDiam,Thresh,Directory_Composite,FileName_Composite,UseWatershed,0) #Last index specifies location in file
    
#Set colormap to grayscale for plotting
plt.gray()

print(nr_nuclei)
#Display Image
plt.figure(figsize=(10,20))
plt.subplot(1,3,1)
plt.title('Preprocessed Image')
plt.imshow(Image_Composite_Gaussian)
plt.subplot (1,3,2)
plt.title('Thresholded Image: ' + str(Thresh))
plt.imshow(Image_Current_T)
plt.subplot(1,3,3)
plt.title('Cell Locations')
plt.jet()
plt.imshow(Image_Current_Cells*(255/Image_Current_Cells.max()))